In [1]:
from IPython.display import display, HTML
from string import Template
from IPython.display import display, HTML
from string import Template
import json
from typing import List
from crawto.Charts.chart_type import Data, DataSet, DataPoint
import jsons
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#data = Data([DataSet("test_Data",list([DataPoint(x=1,y=1),DataPoint(x=2,y=2),DataPoint(x=3,y=3)]), "000000")])

In [3]:
import uuid

class Chart:
    
    def __init__(self):
        self.id = uuid.uuid1()
        self.data = {"datasets":[]}
        self.colorscheme = 'tableauClassic10'
        self.title =  {
                "display": "false",
                "text": ""
            }
        self.scales= {
                "xAxes": [{
                    "display": "true",
                    "scaleLabel": {
                        "display": "false",
                        "labelString": ""
                    }
                }],
                "yAxes": [{
                    "display": "true",
                    "scaleLabel": {
                        "display": "false",
                        "labelString": ""
                    },
                    "type":"linear"
                }]
            }
       
            
    @property
    def html(self):
        html=Template(
        """
        <canvas id= "$id" ></canvas>
        <script>
        requirejs(['https:\\cdnjs.cloudflare.com/ajax/libs/Chart.js/2.8.0/Chart.js'], function(Chart){
            new Chart(document.getElementById("$id"), {
                    type: "$type",
                    data: $data,
                    options: {
                        "responsive": true,
                        "title": $title,
                        "scales" : $scales,
                        tooltips: {
                            callbacks: {
                                label: function(tooltipItem, data) {
                                    var label = data.datasets[tooltipItem.datasetIndex].label;
                                    return label
                               }
                            }
                        }           
                    }
                });
            }); 
        </script>
        """
        )
        html = html.substitute({"data": jsons.dumps(self.data),
                        "id":self.id,
                        "title" :jsons.dumps(self.title),
                        "scales":jsons.dumps(self.scales),
                        "type":self.type,
                       })
        return html
    
    def edit_title(self,text:str,):
        self.title["display"] = "true"
        self.title["text"] = text
        
    def edit_xAxes(self,text:str="",axisIndex:int=0):
        self.scales["xAxes"][axisIndex]["scaleLabel"]["display"] = "true"
        self.scales["xAxes"][axisIndex]["scaleLabel"]["labelString"] = text
        
    def edit_yAxes(self,text:str="",type:str="linear",axisIndex:int=0):
        self.scales["yAxes"][axisIndex]["scaleLabel"]["display"] = "true"
        self.scales["yAxes"][axisIndex]["scaleLabel"]["labelString"] = text
        self.scales["yAxes"][axisIndex]["type"] = type
        
    def __repr__(self):
        return self.html

class ScatterChart(Chart):

    @property
    def type(self):
        return "scatter"
    def add_DataSet(self,label:str,x,y):
        x,y = list(x),list(y)
        if len(x) != len(y):
            raise Exception("x and y columns are not equal in length")
        d = {"label":label,
                  "data": [
                      {"x":float(x[i]), "y":float(y[i])}
                      for i in range(len(x))
                  ],
            }
        self.data["datasets"].append(d)
    
    
        
class BarChart(Chart):
        
    @property
    def type(self):
        return "bar"
    
    def add_DataSet(self,label:str,x, y):
        if "labels" not in self.data.keys():
            self.data["labels"] = list([str(i) for i in x])
        elif list([str(i) for i in x]) != self.data["labels"]:
            raise Exception(f"Already defined the labels for this chart")
        
        self.data["datasets"].append({
            "label":label,
            "data":list(y)
        })

class LineChart(Chart):
    
    
    @property
    def type(self):
        return "line"
    
    def add_DataSet(self,label:str, x,y,fill:str="false"):
        if "labels" not in self.data.keys():
            self.data["labels"] = list([str(i) for i in x])
        elif list([str(i) for i in x]) != self.data["labels"]:
            raise Exception(f"Already defined the labels for this chart")
        
        self.data["datasets"].append({
            "label":label,
            "data":list(y),
            "fill":fill,

        })


In [4]:
x=[-1,0,1]
y = [1,-1,1]

l = LineChart()
l.add_DataSet("t",x,y)
l.add_DataSet("t1" ,x,[2,2,2])
l.edit_title("LineChart")
l.edit_xAxes("x Axis")
l.edit_yAxes("y Axis","linear")

c = ScatterChart()
c.add_DataSet("test",x,y)
c.add_DataSet("t1" ,x,[2,2,2])
c.edit_title("ScatterChart")

b = BarChart()
b.add_DataSet("t",x,y)
b.add_DataSet("t1" ,x,[2,2,2])
b.edit_title("BarChart")

In [5]:
from typing import Union

Chart = Union[ScatterChart]
class Plot:    
    def __init__(self,):
        self.head = """
        <head>
                <script type="application/javascript" 
                src="https:\\cdnjs.cloudflare.com/ajax/libs/require.js/x.y.z/require.js"></script>
                <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/semantic-ui@2.4.2/dist/semantic.min.css">
                <script src="https://code.jquery.com/jquery-3.1.1.min.js"
                  integrity="sha256-hVVnYaiADRTO2PzUGmuLJr8BLUSjGIZsDYGmIJLv2b8="
                  crossorigin="anonymous"></script>
                <script src="https://cdn.jsdelivr.net/npm/semantic-ui@2.4.2/dist/semantic.min.js"></script>
        </head>
        """
        self.body = """
        <body>
        <div class= "ui grid">
        """
    
    def add_column(self,chart:Chart,width:str):
        html = f"<div class = '{width} wide column'>\n"
        html +=chart.html + "\n"
        html +="\n</div>\n"
        self.body+=html
            
    
    def display(self):
        d= self.head+ self.body + "\n</div>\n</body>"
        return HTML(d)
        

In [6]:
p = Plot()

In [7]:
p.add_column(c,"eight")
p.add_column(b,"eight")
p.add_column(l,"eight")

In [8]:
p.display()